In [1]:
!pip install gradio pandas transformers openpyxl

import gradio as gr
import pandas as pd
import random
import tempfile
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the IBM Granite LLM model
model_name = "ibm-granite/granite-3.3-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Global variable to store feedback data
feedback_data = pd.DataFrame(columns=["Issue", "Category", "Timestamp"])

# Function to generate responses from the model with full output
def generate_response(prompt, max_length=1000):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Smart Waste Management Tips
def waste_management(item):
    prompt = f"Provide detailed disposal instructions for {item}, including recycling options if applicable."
    return generate_response(prompt)

# Energy Saving Suggestions
def energy_saving(habits):
    prompt = f"Based on these energy habits: {habits}. Provide comprehensive energy-efficient practices with explanations."
    return generate_response(prompt)

# Citizen Feedback Collector with Excel integration and download
def collect_feedback(issue, category, feedback_file=None):
    global feedback_data

    # Add new feedback to dataframe
    new_entry = pd.DataFrame({
        "Issue": [issue],
        "Category": [category],
        "Timestamp": [pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")]
    })

    feedback_data = pd.concat([feedback_data, new_entry], ignore_index=True)

    base_response = f"✅ Feedback received!\n\nIssue: {issue}\nCategory: {category}\n\nTotal feedback entries: {len(feedback_data)}"

    if feedback_file:
        try:
            # Read uploaded Excel file
            df = pd.read_excel(feedback_file.name)

            # Merge with existing data
            if set(df.columns) == {"Issue", "Category", "Timestamp"}:
                feedback_data = pd.concat([feedback_data, df], ignore_index=True).drop_duplicates()
                base_response += f"\n\nAdded {len(df)} entries from Excel file"
            else:
                base_response += "\n\nNote: Excel file format didn't match expected columns (Issue, Category, Timestamp)"
        except Exception as e:
            base_response += f"\n\n⚠ Excel processing error: {str(e)}"

    # Create downloadable CSV
    with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp:
        feedback_data.to_csv(tmp.name, index=False)
        download_path = tmp.name

    return base_response, download_path

# Green Challenge Generator
def green_challenge():
    challenges = [
        "Avoid plastic bags today. Bring reusable bags when shopping.",
        "Take a 30-minute walk instead of driving for short distances.",
        "Turn off all non-essential lights and electronics for one hour this evening.",
        "Avoid using disposable coffee cups. Bring your own reusable cup.",
        "Spend 15 minutes collecting litter in your neighborhood.",
        "Use only reusable water bottles today - no single-use plastic bottles.",
        "Visit a local farmer's market and purchase one locally grown item."
    ]
    return random.choice(challenges)

# Policy Document Summarization
def summarize_policy(document):
    prompt = f"Please provide a comprehensive summary of this policy document: {document}"
    return generate_response(prompt)

# KPI Forecasting
def forecast_kpi(data):
    if data is None:
        return "No data uploaded."

    try:
        df = pd.read_csv(data.name)

        if 'usage' not in df.columns:
            return "Error: The uploaded CSV must contain a 'usage' column."

        avg_usage = df['usage'].mean()
        forecast = avg_usage * 1.05

        return (f"Water Usage Forecast:\nCurrent average: {avg_usage:.2f} units\n"
               f"Forecasted next period: {forecast:.2f} units\n\n"
               f"Historical data points: {len(df)}\n"
               f"Date range: {df['date'].min()} to {df['date'].max()}")
    except Exception as e:
        return f"Error processing data: {str(e)}"

# Eco Tips Generator
def eco_tips(keyword):
    prompt = f"Provide 5 detailed sustainability tips related to {keyword}."
    return generate_response(prompt)

# Anomaly Detection in Energy Data
def detect_anomaly(data):
    if data is None:
        return "No data uploaded."

    try:
        df = pd.read_csv(data.name)

        if 'usage' not in df.columns:
            return "Error: The uploaded CSV must contain a 'usage' column."

        mean = df['usage'].mean()
        std = df['usage'].std()
        threshold = mean + 2 * std

        anomalies = df[df['usage'] > threshold]

        if anomalies.empty:
            return (f"No significant anomalies detected.\nMean usage: {mean:.2f}\n"
                   f"Standard deviation: {std:.2f}")
        else:
            return (f"Anomalies Detected:\nTotal: {len(anomalies)}\nThreshold: > {threshold:.2f}\n\n"
                   f"Anomalous values:\n{anomalies[['date', 'usage']].to_string(index=False)}")
    except Exception as e:
        return f"Error processing data: {str(e)}"

# Chat Assistant with full output
def chat_assistant(question):
    prompt = f"User asked: {question}\n\nProvide a detailed, comprehensive response with actionable sustainability strategies."
    return generate_response(prompt, max_length=1500)

# Create Gradio Interface with enhanced UI
with gr.Blocks(css="""
    /* Main App Styles */
    .gradio-container {
        background: linear-gradient(135deg, #e6f2ff, #f0f8ff) !important;
    }

    /* Header Styles */
    .gradio-header {
        background: linear-gradient(90deg, #0077b6, #0096c7) !important;
        padding: 1.5rem !important;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1) !important;
        border-radius: 0 0 12px 12px !important;
    }

    .gradio-header h1 {
        color: white !important;
        font-size: 2.2rem !important;
        text-align: center !important;
        margin: 0 !important;
        font-weight: 600 !important;
        text-shadow: 1px 1px 3px rgba(0, 0, 0, 0.2) !important;
    }

    /* Tab Navigation */
    .tab-nav {
        background: transparent !important;
        border-bottom: none !important;
    }

    .tab-button {
        background: #f8f9fa !important;
        color: #495057 !important;
        border-radius: 8px 8px 0 0 !important;
        padding: 12px 24px !important;
        margin-right: 4px !important;
        font-weight: 500 !important;
        transition: all 0.3s ease !important;
        border: 1px solid #dee2e6 !important;
    }

    .tab-button:hover {
        background: #e9ecef !important;
        transform: translateY(-2px) !important;
    }

    .tab-button.selected {
        background: #0077b6 !important;
        color: white !important;
        border-color: #0077b6 !important;
        box-shadow: 0 4px 8px rgba(0, 119, 182, 0.2) !important;
    }

    /* Content Area */
    .tab-content {
        background: white !important;
        border-radius: 8px !important;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1) !important;
        padding: 24px !important;
        margin-top: 16px !important;
        min-height: 500px;
    }

    /* Button Styles */
    button {
        background: #0077b6 !important;
        color: white !important;
        border: none !important;
        border-radius: 8px !important;
        padding: 12px 24px !important;
        font-weight: 500 !important;
        cursor: pointer !important;
        transition: all 0.3s ease !important;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1) !important;
    }

    button:hover {
        background: #0096c7 !important;
        transform: translateY(-2px) !important;
        box-shadow: 0 4px 8px rgba(0, 119, 182, 0.3) !important;
    }

    button:active {
        transform: translateY(0) !important;
    }

    /* Input Styles */
    .gr-textarea, .gr-textbox, .gr-input {
        background: #f8f9fa !important;
        border: 1px solid #ced4da !important;
        border-radius: 8px !important;
        padding: 12px !important;
        transition: all 0.3s ease !important;
    }

    .gr-textarea:focus, .gr-textbox:focus, .gr-input:focus {
        border-color: #0077b6 !important;
        box-shadow: 0 0 0 3px rgba(0, 119, 182, 0.2) !important;
        outline: none !important;
    }

    /* Output Styles */
    .output-text, .gr-markdown {
        background: #f8f9fa !important;
        border-radius: 8px !important;
        padding: 16px !important;
        border-left: 4px solid #0077b6 !important;
    }

    /* Label Styles */
    label {
        font-weight: 500 !important;
        margin-bottom: 8px !important;
        color: #495057 !important;
    }

    /* File Upload Styles */
    .file-upload {
        border: 2px dashed #adb5bd !important;
        border-radius: 8px !important;
        padding: 24px !important;
        background: #f8f9fa !important;
        transition: all 0.3s ease !important;
    }

    .file-upload:hover {
        border-color: #0077b6 !important;
        background: #e9ecef !important;
    }

    /* Download Button */
    .download-button {
        background: #28a745 !important;
    }

    .download-button:hover {
        background: #218838 !important;
    }

    /* Dark/Light Mode Toggle */
    .toggle-container {
        position: absolute;
        top: 1rem;
        right: 1rem;
    }

    /* Responsive Adjustments */
    @media (max-width: 768px) {
        .gradio-header h1 {
            font-size: 1.8rem !important;
        }

        .tab-button {
            padding: 8px 16px !important;
            font-size: 0.9rem !important;
        }
    }
""") as app:

    # App Header
    gr.Markdown("""
    <div style="text-align: center; padding: 2rem 0;">
        <h1 style="font-size: 2.5rem; color: #0077b6; margin-bottom: 0.5rem;">
            <span style="font-weight: 300;">Sustainable</span>
            <span style="font-weight: 700;">Smart City</span>
            <span style="font-weight: 300;">Assistant</span>
        </h1>
        <p style="color: #6c757d; font-size: 1.1rem;">
            AI-powered urban sustainability toolkit with complete output visibility
        </p>
    </div>
    """)

    with gr.Tabs():
        # Tab 1: Waste Management
        with gr.Tab("♻ Waste Management"):
            with gr.Row():
                with gr.Column():
                    item_input = gr.Textbox(label="Item to dispose of",
                                          placeholder="e.g., plastic bottles, batteries",
                                          elem_classes=["styled-input"])
                    waste_btn = gr.Button("Get Disposal Guide",
                                         elem_classes=["primary-button"])
                with gr.Column():
                    waste_output = gr.Textbox(label="Disposal Instructions",
                                            lines=20,
                                            interactive=False,
                                            elem_classes=["styled-output"])
            waste_btn.click(waste_management, inputs=item_input, outputs=waste_output)

        # Tab 2: Energy Savings
        with gr.Tab("⚡ Energy Savings"):
            with gr.Row():
                with gr.Column():
                    habits_input = gr.Textbox(label="Describe your energy habits",
                                            lines=3,
                                            elem_classes=["styled-input"])
                    energy_btn = gr.Button("Get Recommendations",
                                         elem_classes=["primary-button"])
                with gr.Column():
                    energy_output = gr.Textbox(label="Energy Saving Tips",
                                             lines=20,
                                             interactive=False,
                                             elem_classes=["styled-output"])
            energy_btn.click(energy_saving, inputs=habits_input, outputs=energy_output)

        # Tab 3: Feedback Collection
        with gr.Tab("📝 Feedback Collection"):
            with gr.Row():
                with gr.Column():
                    issue_input = gr.Textbox(label="Issue description",
                                           elem_classes=["styled-input"])
                    category_input = gr.Textbox(label="Category (e.g., waste, water)",
                                              elem_classes=["styled-input"])
                    feedback_file = gr.File(label="Upload Excel feedback log (optional)",
                                          file_types=[".xlsx"],
                                          elem_classes=["file-upload"])
                    feedback_btn = gr.Button("Submit Feedback",
                                          elem_classes=["primary-button"])
                with gr.Column():
                    feedback_output = gr.Textbox(label="Feedback Status",
                                               lines=10,
                                               interactive=False,
                                               elem_classes=["styled-output"])
                    download_button = gr.DownloadButton("Download All Feedback",
                                                       visible=False,
                                                       elem_classes=["download-button"])

            def handle_feedback(issue, category, feedback_file):
                response, download_path = collect_feedback(issue, category, feedback_file)
                return response, gr.DownloadButton("Download All Feedback",
                                                 visible=True,
                                                 value=download_path,
                                                 elem_classes=["download-button"])

            feedback_btn.click(
                handle_feedback,
                inputs=[issue_input, category_input, feedback_file],
                outputs=[feedback_output, download_button]
            )

        # Tab 4: Green Challenge
        with gr.Tab("🌿 Daily Challenge"):
            with gr.Row():
                challenge_btn = gr.Button("Get Today's Eco Challenge",
                                       elem_classes=["primary-button"])
                challenge_output = gr.Textbox(label="Your Challenge",
                                            interactive=False,
                                            elem_classes=["styled-output"])
            challenge_btn.click(green_challenge, outputs=challenge_output)

        # Tab 5: Policy Summarizer
        with gr.Tab("📑 Policy Summary"):
            with gr.Row():
                with gr.Column():
                    policy_input = gr.Textbox(label="Policy text",
                                            lines=5,
                                            elem_classes=["styled-input"])
                    policy_btn = gr.Button("Summarize",
                                         elem_classes=["primary-button"])
                with gr.Column():
                    policy_output = gr.Textbox(label="Summary",
                                             lines=20,
                                             interactive=False,
                                             elem_classes=["styled-output"])
            policy_btn.click(summarize_policy, inputs=policy_input, outputs=policy_output)

        # Tab 6: KPI Forecasting
        with gr.Tab("📊 KPI Forecast"):
            with gr.Row():
                with gr.Column():
                    kpi_input = gr.File(label="Upload CSV with usage data",
                                      elem_classes=["file-upload"])
                    kpi_btn = gr.Button("Generate Forecast",
                                      elem_classes=["primary-button"])
                with gr.Column():
                    kpi_output = gr.Textbox(label="Forecast Results",
                                          lines=20,
                                          interactive=False,
                                          elem_classes=["styled-output"])
            kpi_btn.click(forecast_kpi, inputs=kpi_input, outputs=kpi_output)

        # Tab 7: Eco Tips
        with gr.Tab("💡 Eco Tips"):
            with gr.Row():
                with gr.Column():
                    keyword_input = gr.Textbox(label="Topic (e.g., recycling, energy)",
                                             elem_classes=["styled-input"])
                    tips_btn = gr.Button("Get Tips",
                                       elem_classes=["primary-button"])
                with gr.Column():
                    tips_output = gr.Textbox(label="Sustainability Tips",
                                           lines=20,
                                           interactive=False,
                                           elem_classes=["styled-output"])
            tips_btn.click(eco_tips, inputs=keyword_input, outputs=tips_output)

        # Tab 8: Anomaly Detection
        with gr.Tab("🔍 Anomaly Detection"):
            with gr.Row():
                with gr.Column():
                    anomaly_input = gr.File(label="Upload energy data CSV",
                                          elem_classes=["file-upload"])
                    anomaly_btn = gr.Button("Analyze Data",
                                          elem_classes=["primary-button"])
                with gr.Column():
                    anomaly_output = gr.Textbox(label="Analysis Results",
                                              lines=20,
                                              interactive=False,
                                              elem_classes=["styled-output"])
            anomaly_btn.click(detect_anomaly, inputs=anomaly_input, outputs=anomaly_output)

        # Tab 9: Chat Assistant
        with gr.Tab("💬 Chat Assistant"):
            with gr.Row():
                with gr.Column():
                    chat_input = gr.Textbox(label="Your sustainability question",
                                          lines=3,
                                          elem_classes=["styled-input"])
                    chat_btn = gr.Button("Ask Assistant",
                                       elem_classes=["primary-button"])
                with gr.Column():
                    chat_output = gr.Textbox(label="Assistant Response",
                                           lines=25,
                                           interactive=False,
                                           elem_classes=["styled-output"])
            chat_btn.click(chat_assistant, inputs=chat_input, outputs=chat_output)

# Launch the app
app.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af0387681482e93e71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
